In [1]:
import numpy as np
#from scipy.signal import butter, filtfilt
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
#import seaborn as sns
#sns.set_context("notebook", font_scale=1.3,
#                rc={'lines.linewidth': 1.5, 'lines.markersize': 14, 'axes.titlesize': 'x-large'})
#matplotlib.rc('legend', numpoints=1, fontsize=14)
#import glob
import sys, os
sys.path.insert(1, r'./Functions')
# IPython widgets:
from IPython.display import display
import ipywidgets
from ipywidgets import FloatProgress, interactive
import seaborn as sns
from scipy.signal import detrend

In [2]:
#!pip install version_information
%load_ext version_information
%version_information numpy, scipy, pandas, matplotlib, seaborn

Software versions
Python 3.8.8 64bit [MSC v.1916 64 bit (AMD64)]
IPython 7.22.0
OS Windows 10 10.0.22621 SP0
numpy 1.20.1
scipy 1.6.2
pandas 1.2.4
matplotlib 3.3.4
seaborn 0.11.1
Thu Mar 30 14:00:20 2023 Hora oficial do Brasil

In [3]:
# GitHub URL:
#path2 = 'C:\Users\ferna\Downloads\IC- dados\Dados'
# local directory:
path2 = r'C:\\Users\\ferna\\Downloads\\IC- dados\\Dados'
fname = os.path.join(path2, 'PDSinfo.txt')
PDSinfo = pd.read_csv(fname, sep='\t', header=0, index_col=None, engine='c', encoding='utf-8')
print(fname)
print("Information of %s subjects loaded (%s rows, %s columns)."
      %(len(pd.unique(PDSinfo.Subject)), PDSinfo.shape[0], PDSinfo.shape[1]))

C:\\Users\\ferna\\Downloads\\IC- dados\\Dados\PDSinfo.txt
Information of 49 subjects loaded (588 rows, 29 columns).


In [4]:
a=PDSinfo[PDSinfo['Subject']==44].index.tolist()
e=PDSinfo[PDSinfo['Subject']==44].index.tolist()[2]
ei = PDSinfo[PDSinfo['Subject']==44]
PDSinfo.memory_usage()

Index                 128
Trial                4704
Subject              4704
Vision               4704
Surface              4704
Rep                  4704
Age                  4704
AgeGroup             4704
Gender               4704
Height               4704
Mass                 4704
BMI                  4704
FootLen              4704
DominantLeg          4704
Nationality          4704
SkinColor            4704
Ystudy               4704
Footwear             4704
Illness              4704
Illness2             4704
Nmedication          4704
Medication           4704
Ortho-Prosthesis     4704
Ortho-Prosthesis2    4704
Disability           4704
Disability2          4704
Falls12m             4704
PhysicalActivity     4704
Sequence             4704
Date                 4704
dtype: int64

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
fs = 100 # Hz
COPX = np.zeros((49, 60*fs-30))
COPZ = np.zeros((49, 60*fs-30))

COPXreal = np.zeros((49, 60*fs-30))
COPYreal = np.zeros((49, 60*fs-30))
COPZreal = np.zeros((49, 60*fs-30))

d1x = np.zeros((49, 60*fs-30))
d2x = np.zeros((49, 60*fs-30))

d1z = np.zeros((49, 60*fs-30))
d2z = np.zeros((49, 60*fs-30))

matrizangx = np.zeros((49, 60*fs-30))
matrizangz = np.zeros((49, 60*fs-30))

alpha = np.zeros((49, 60*fs-30))

COGX = np.zeros((49, 60*fs-30))
COGY = np.zeros((49,60*fs-30))
COGZ = np.zeros((49, 60*fs-30))  

for s in range (1,50):
    #a = PDSinfo[PDSinfo['Subject']==s].index.tolist()[2]
    ei = PDSinfo[PDSinfo['Subject']==s].index.tolist()
    ih=[]
    for l in range(0,len(ei)):
        a = ei[l]
        
        fname_grf = os.path.join(path2, PDSinfo.Trial[a] + 'grf' + '.txt') 
        fname_mkr = os.path.join(path2, PDSinfo.Trial[a] + 'mkr' + '.txt')
        grf = pd.read_csv(fname_grf, delimiter='\t', header=0, engine='c')
        mkr = pd.read_csv(fname_mkr, delimiter='\t', header=0, engine='c')
        #print(mkr)
        #print(np.where(np.isnan(mkr)))
        #i=mkr.dropna(axis=0, how ='all')
        
        COGX[s-1,:] = mkr['COG_X'].values[30:]
        
        COGY[s-1,:]=mkr['COG_Y'].values[30:]
        COGZ[s-1,:]=mkr['COG_Z'].values[30:]
        
        #print(np.where(np.isnan(COGX[s-1,:])))
        
        mass=PDSinfo['Mass']
        d = np.sqrt(COGX[s-1,:]**2+COGY[s-1,:]**2+COGZ[s-1,:]**2)
        angulox = np.arcsin(COGX[s-1,:]/d)
        anguloz= np.arcsin(COGZ[s-1,:]/d)
        sinx=np.sin(angulox)
        sinz=np.sin(anguloz)
        cosx=np.cos(angulox)
        cosz=np.cos(anguloz)
        tanx=np.tan(angulox)
        tanz=np.tan(anguloz)
        matrizangx[s-1,:] = angulox
        matrizangz[s-1,:] = anguloz
    
    
        Jb=(4/3)*(mass.values[s])*((d)**2)
        t=grf['Time']
        dt=t[1]-t[0]

        d1x[s-1,:]=np.gradient(matrizangx[s-1,:],dt)
        d1z[s-1,:]=np.gradient(matrizangz[s-1,:],dt)

        d2x[s-1,:]=np.gradient(d1x[s-1,:],dt)
        d2z[s-1,:]=np.gradient(d1z[s-1,:],dt)

        COPX[s-1,:] = d*sinx + Jb/(mass.values[s]*9.8)*(sinx*d1x[s-1,:]**2-cosx*d2x[s-1,:])
        COPZ[s-1,:] = d*sinz + Jb/(mass.values[s]*9.8)*(sinz*d1z[s-1,:]**2-cosz*d2z[s-1,:])


        COPXreal[s-1,:]=grf['COPNET_X'].values[30:]
        COPZreal[s-1,:]=grf['COPNET_Z'].values[30:]



        x= np.vstack((COPX[[s-1],:],d2x[[s-1],:]))
        y= COPXreal[s-1,:]

        model=LinearRegression(fit_intercept=False)
        model.fit(x.T,y)


        split=train_test_split(COGX,COPXreal,test_size=9)
        COGXtrain = split[0]
        COPXtrain = split[1]
        COGXtest = split[2]
        COPXtest = split[3]

#print(COGXtrain)
#for i in COGXtrain:
 #   i=0
#print(COGXtrain[i])
    
    
    
    
    #print(model.coef_) 
    
    #def reg_m(y, x):
        #ones = np.ones(len(x[0]))
        #X = sm.add_constant(np.column_stack((x[0], ones)))
        #for oi in x[1:]:
            #X = sm.add_constant(np.column_stack((oi, X)))
            #results = sm.OLS(y, x).fit()
            #return results
    #print(model.coef_)
    #print(reg_m(y, x.T).summary())
 
    #print(model.coef_)
    #x= (np.array([COPX[s-1,:],d2x[s-1,:]]))
    #y= COPXreal[s-1,:]
    #model=LinearRegression()
    #model.fit(x.T,y)
    #print(model.coef_) 
    #Histogramas
    #plt.figure()
    #hist=plt.hist(model.coef_[0], bins=50)
    #plt.title('Histograms of alpha'.format(s))
    #plt.show()


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
fs = 100 # Hz
COPX = np.zeros((49, 60*fs-30))
COPZ = np.zeros((49, 60*fs-30))

COPXreal = np.zeros((49, 60*fs-30))
COPYreal = np.zeros((49, 60*fs-30))
COPZreal = np.zeros((49, 60*fs-30))

d1x = np.zeros((49, 60*fs-30))
d2x = np.zeros((49, 60*fs-30))

d1z = np.zeros((49, 60*fs-30))
d2z = np.zeros((49, 60*fs-30))

matrizangx = np.zeros((49, 60*fs-30))
matrizangz = np.zeros((49, 60*fs-30))

alpha = np.zeros((49, 60*fs-30))

COGX = np.zeros((49, 60*fs-30))
COGY = np.zeros((49,60*fs-30))
COGZ = np.zeros((49, 60*fs-30))  


COGYnovo = np.zeros((49, 60*fs-30))
COGZnovo = np.zeros((49, 60*fs-30))
COGXnovo = np.zeros((49, 60*fs-30))

for s in range (1,50):
    #a = PDSinfo[PDSinfo['Subject']==s].index.tolist()[2]
    ei = PDSinfo[PDSinfo['Subject']==s].index.tolist()
    ih=[]
    for l in range(0,len(ei)):
        a = ei[l]
        
        fname_grf = os.path.join(path2, PDSinfo.Trial[a] + 'grf' + '.txt') 
        fname_mkr = os.path.join(path2, PDSinfo.Trial[a] + 'mkr' + '.txt')
        grf = pd.read_csv(fname_grf, delimiter='\t', header=0, engine='c')
        mkr = pd.read_csv(fname_mkr, delimiter='\t', header=0, engine='c')
        #print(mkr)
        #print(np.where(np.isnan(mkr)))
        #i=mkr.dropna(axis=0, how ='all')
        
        
        COGX[s-1,:] = mkr['COG_X'].values[30:]
        
        COGY[s-1,:]=mkr['COG_Y'].values[30:]
        COGZ[s-1,:]=mkr['COG_Z'].values[30:]
        
        #print((COGX[s-1,:]).shape)
        
        
        COGXnovo = (COGX[s-1,:])[~(np.isnan(COGX[s-1,:]))]
        COGYnovo = (COGY[s-1,:])[np.logical_not(np.isnan(COGY[s-1,:]))]
        COGZnovo =(COGZ[s-1,:])[np.logical_not(np.isnan(COGZ[s-1,:]))]
        COGXnovo = (COGX[s-1,:])[np.logical_not(np.isnan(COGX[s-1,:]))]
        #np.delete(COGXnovo,32,axis=0)
        
        if ((COGXnovo).size)!=0:
            
            mass=PDSinfo['Mass']
            d = np.sqrt(COGXnovo[s-1,]**2+COGYnovo[s-1,]**2+COGZnovo[s-1,]**2)
            angulox= np.arcsin(COGXnovo[s-1,]/d)
            anguloz= np.arcsin(COGZnovo[s-1,]/d)
            sinx=np.sin(angulox)
            sinz=np.sin(anguloz)
            cosx=np.cos(angulox)
            cosz=np.cos(anguloz)
            tanx=np.tan(angulox)
            tanz=np.tan(anguloz)
            matrizangx[s-1,] = angulox
            matrizangz[s-1,] = anguloz


            Jb=(4/3)*(mass.values[s])*((d)**2)
            t=grf['Time']
            dt=t[1]-t[0]

            d1x[s-1,]=np.gradient(matrizangx[s-1,],dt)
            d1z[s-1,]=np.gradient(matrizangz[s-1,],dt)

            d2x[s-1,]=np.gradient(d1x[s-1,],dt)
            d2z[s-1,]=np.gradient(d1z[s-1,],dt)

            COPX[s-1,] = d*sinx + Jb/(mass.values[s]*9.8)*(sinx*d1x[s-1,]**2-cosx*d2x[s-1,])
            COPZ[s-1] = d*sinz + Jb/(mass.values[s]*9.8)*(sinz*d1z[s-1,]**2-cosz*d2z[s-1,])


            COPXreal[s-1,]=grf['COPNET_X'].values[30:]
            COPZreal[s-1]=grf['COPNET_Z'].values[30:]



            x= np.vstack((COPX[s-1,],d2x[s-1,]))
            y= COPXreal[s-1,]

            model=LinearRegression(fit_intercept=False)
            model.fit(x.T,y)


            split=train_test_split(COGX,COPXreal,test_size=9)
            COGXtrain = split[0]
            COPXtrain = split[1]
            COGXtest = split[2]
            COPXtest = split[3]
        

#print(COGXtrain)
#for i in COGXtrain:
 #   i=0
#print(COGXtrain[i])
    
    
    
    
    #print(model.coef_) 
    
    #def reg_m(y, x):
        #ones = np.ones(len(x[0]))
        #X = sm.add_constant(np.column_stack((x[0], ones)))
        #for oi in x[1:]:
            #X = sm.add_constant(np.column_stack((oi, X)))
            #results = sm.OLS(y, x).fit()
            #return results
    #print(model.coef_)
    #print(reg_m(y, x.T).summary())
 
    #print(model.coef_)
    #x= (np.array([COPX[s-1,:],d2x[s-1,:]]))
    #y= COPXreal[s-1,:]
    #model=LinearRegression()
    #model.fit(x.T,y)
    #print(model.coef_) 
    #Histogramas
    #plt.figure()
    #hist=plt.hist(model.coef_[0], bins=50)
    #plt.title('Histograms of alpha'.format(s))
    #plt.show()


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
fs = 100 # Hz
import pickle 
#file = open('important', 'wb')

COPX = np.zeros((49, 60*fs-30))
COPZ = np.zeros((49, 60*fs-30))

COPXreal = np.zeros((0, 60*fs-30))
COPYreal = np.zeros((0, 60*fs-30))
COPZreal = np.zeros((0, 60*fs-30))

d1x = np.zeros((0, 60*fs-30))
d2x = np.zeros((0, 60*fs-30))

d1z = np.zeros((0, 60*fs-30))
d2z = np.zeros((0, 60*fs-30))

matrizangx = np.zeros((0, 60*fs-30))
matrizangz = np.zeros((0, 60*fs-30))

alpha = np.zeros((0, 60*fs-30))


ASISX = np.empty((0, 60*fs-30))
ASISY = np.empty((0,60*fs-30))
ASISZ = np.empty((0, 60*fs-30))

ASISXnovo = np.empty((0, 60*fs-30))
ASISYnovo = np.empty((0, 60*fs-30))
ASISZnovo = np.empty((0, 60*fs-30))

d = np.zeros((0, 60*fs-30))


angulox = np.zeros((0, 60*fs-30))
anguloz= np.zeros((0, 60*fs-30))

matrizangx = np.zeros((0, 60*fs-30))
matrizangz = np.zeros((0, 60*fs-30))



subject = np.zeros((0,1))
j=0



for s in range (1,50):
    
    #a = PDSinfo[PDSinfo['Subject']==s].index.tolist()[2]
    
    ei = PDSinfo[PDSinfo['Subject']==s].index.tolist()
    
    
    for i in range(0,len(ei)):
        a = ei[i]
        fname_grf = os.path.join(path2, PDSinfo.Trial[a] + 'grf' + '.txt') 
        fname_mkr = os.path.join(path2, PDSinfo.Trial[a] + 'mkr' + '.txt')
        grf = pd.read_csv(fname_grf, delimiter='\t', header=0, engine='c')
        mkr = pd.read_csv(fname_mkr, delimiter='\t', header=0, engine='c')
        colunas = mkr.columns
        #print(mkr.memory_usage())
        
        
        asisx=(mkr['R.ASIS_X'].values[30:])
        asisy=(mkr['R.ASIS_Y'].values[30:])
        asisz=(mkr['R.ASIS_Z'].values[30:])
        
        
        
        if (any(np.isnan(asisx))) != 'True' :
            ASISX = np.vstack((ASISX,asisx))
            ASISY = np.vstack((ASISY,asisy))
            ASISZ = np.vstack((ASISZ,asisz))
        
        
    
            
            mass=PDSinfo['Mass']
            d =np.vstack((d,np.sqrt(ASISX[-1,:]**2+ASISY[-1,:]**2+ASISZ[-1,:]**2)))
            angulox = np.vstack((angulox,np.arcsin(ASISX[-1,:]/d[-1,:])))
            anguloz= np.vstack((anguloz,np.arcsin(ASISZ[-1,:]/d[-1,:])))
            sinx=np.sin(angulox[-1,:])
            sinz=np.sin(anguloz[-1,:])
            cosx=np.cos(angulox[-1,:])
            cosz=np.cos(anguloz[-1,:])
            tanx=np.tan(angulox[-1,:])
            tanz=np.tan(anguloz[-1,:])
            matrizangx = np.vstack((matrizangx,angulox[-1,:]))
            matrizangz = np.vstack((matrizangz,anguloz[-1,:])) 
            

            Jb=(4/3)*(mass.values[s])*((d[-1,:])**2)
            t=grf['Time']
            dt=t[1]-t[0]
            

            d1x = np.vstack((d1x,np.gradient(matrizangx[-1,:],dt)))
            d1z = np.vstack((d1z,np.gradient(matrizangz[-1,:],dt)))

            d2x = np.vstack((d2x,np.gradient(d1x[-1,:],dt)))
            d2z = np.vstack((d2z,np.gradient(d1z[-1,:],dt)))

            COPX = np.vstack((COPX,d*sinx + Jb/(mass.values[s]*9.8)*(sinx*d1x[-1,:]**2-cosx*d2x[-1,:]))) 
            COPZ = np.vstack((COPZ,d*sinz + Jb/(mass.values[s]*9.8)*(sinz*d1z[-1,:]**2-cosz*d2z[-1,:])))




            COPXreal=np.vstack((COPXreal,grf['COPNET_X'].values[30:]))
            COPZreal=np.vstack((COPZreal,grf['COPNET_Z'].values[30:]))

            subject = np.vstack((subject, [s]))
        
        #ASISXnovo = np.vstack((ASISXnovo,(ASISX[-1,:])[np.logical_not(np.isnan(ASISX[-1,:]))]))
        #ASISYnovo = np.vstack((ASISYnovo,(ASISY[-1,:])[np.logical_not(np.isnan(ASISY[-1,:]))]))
        #ASISZnovo = np.vstack((ASISZnovo,(ASISZ[-1,:])[np.logical_not(np.isnan(ASISZ[-1,:]))]))
        
        
        #if (ASISXnovo.size)!=0:
       

filename = 'COPX.pkl'

            
pickle.dump(COPXreal,filename)

filename = 'COPZ.pkl'

pickle.dump(COPZreal,filename)


In [ ]:
indices = np.arange(1,50).reshape(-1,1)
np.random.shuffle(indices)
train = indices[:40]
test = indices[40:]

masstrain = mass.values[train]
masstest = mass.values[test]

np.in1d(subject,train)
print(COGX.shape)
print(COGXnovo.shape)

ASISXtrain = ASISXnovo[np.in1d(subject,train),]
COGXtrain = COGXnovo[np.in1d(subject,train),]
COGXtest = COGXnovo[np.in1d(subject,test),]
ASISXtest = ASISXnovo[np.in1d(subject,test),]
COPXtrain = COPXnovo[np.in1d(subject,train),]
COPXtest = COPXnovo[np.in1d(subject,test),]

ASISYtrain = ASISYnovo[np.in1d(subject,train),]
COGYtrain = COGYnovo[np.in1d(subject,train),]
COGYtest = COGYnovo[np.in1d(subject,test),]
ASISYtest = ASISYnovo[np.in1d(subject,test),]

ASISZtrain = ASISZnovo[np.in1d(subject,train),]
ASISZtest = ASISZnovo[np.in1d(subject,test),]
COPZtrain = COPZnovo[np.in1d(subject,train),]
COPZtest = COPZnovo[np.in1d(subject,test),]
COGZtrain = COGZnovo[np.in1d(subject,train),]
COGZtest = COGZnovo[np.in1d(subject,test),]



In [ ]:
def newdist(paramt,x,y,z):
    from sklearn.model_selection import train_test_split
    from scipy.optimize import minimize, rosen, rosen_der
    from sklearn.metrics import mean_squared_error
    dest = np.zeros((60*fs-30))
    d = np.zeros((60*fs-30))

    for i in range(0,len(x)):
        
        delta,mi = paramt
        
        d[i] = np.mean(np.sqrt(x[i]**2+y[i]**2+z[i]**2))
        
        dest[i] = delta * d[i] + mi
       
    return dest


In [ ]:
#for i in range(0,len(COGXtrain)):
        
        #d= np.sqrt(COGXtrain[i]**2+COGYtrain[i]**2+COGZtrain[i]**2)
        
def meansquarerrordist(paramt,d,ASISXtrain,ASISYtrain,ASISZtrain):
    d = np.zeros((60*fs-30))
    from sklearn.metrics import mean_squared_error
    ytrue = np.zeros((40,60*fs-30))
    ypredicted = np.zeros((40,60*fs-30))
    for i in range(0,len(COGXtrain)):
        
        d[i] = np.mean(np.sqrt(COGXtrain[i]**2+COGYtrain[i]**2+COGZtrain[i]**2))
        
        ytrue[i] = d[i]
        
        ypredicted[i] = newdist(paramt,ASISXtrain,ASISYtrain,ASISZtrain)
    
    
    
        
    return mean_squared_error(ytrue, ypredicted)


In [ ]:
from sklearn.model_selection import train_test_split
#from scipy import optmize
from scipy.optimize import minimize, rosen, rosen_der
first_guess = [1,1]
res = minimize(meansquarerrordist, first_guess,(d,COGXtrain,COGYtrain,COGZtrain))
coeficientes = res.x
delta=coeficientes[0]
mi=coeficientes[1]
print(delta)
print(mi)

In [ ]:
treino = newdist(coeficientes,COGXtrain,COGYtrain,COGZtrain)
teste = newdist(coeficientes,COGXtest,COGYtest,COGZtest)

Regressão linear distância

In [ ]:
def centerofpressure(paramt, M,x,y,z):
    from sklearn.model_selection import train_test_split
    from scipy.optimize import minimize, rosen, rosen_der
    from sklearn.metrics import mean_squared_error
    COPXES = np.zeros((len(x), 60*fs-30))
    COPXESorig =np.zeros((len(x), 60*fs-30))
    dest =np.zeros((60*fs-30))
    for i in range(0,len(x)):
        
        A,alpha,beta, gama= paramt
        
        d[i] = np.mean(np.sqrt(x[i]**2+y[i]**2+z[i]**2))
        dest[i] = delta * d[i] + mi
        angulox = np.arcsin(np.tanh(x[i]/dest[i]))
        anguloz = np.arcsin(np.tanh(z[i]/dest[i]))
        sinx=np.sin(angulox)
        sinz=np.sin(anguloz)
        cosx=np.cos(angulox)
        cosz=np.cos(anguloz)
        tanx=np.tan(angulox)
        tanz=np.tan(anguloz)
        matrizangx[i,:] = angulox
        matrizangz[i,:] = anguloz

        
        
        
        
        

        sennovo = (sinx)*A + ((1-A**2)**1/2)*(cosx)

        cosnovo = (cosx)*A - ((1-A**2)**1/2)*(sinx)
        

        

        Jb=(4/3)*(M[i])*((dest)**2)
        t=grf['Time']
        dt=t[1]-t[0]

        d1x[i,:]=np.gradient(matrizangx[i,:],dt)
        d1z[i,:]=np.gradient(matrizangz[i,:],dt)



        d2x[i,:]=np.gradient(d1x[i,:],dt)
        d2z[i,:]=np.gradient(d1z[i,:],dt)

        COPX[i,:] = dest*sennovo + Jb/(M[i]*9.8)*(sennovo*d1x[i,:]**2-cosnovo*d2x[i,:])
        COPZ[i,:] = dest*sinz + Jb/(M[i]*9.8)*(sinz*d1z[i,:]**2-cosz*d2z[i,:])
        
        COPXES[i,:] = alpha*(dest*np.tanh(sennovo) + ((Jb/(M[i]*9.8))*np.tanh(sennovo)*d1x[i,:]**2)) + d2x[i,:]*(-alpha*np.tanh(cosnovo)*Jb/(M[i]*9.8)+beta) + gama
        COPXESorig[i,:]= COPX[i,:]
        
    print(dest)
          
    
    return COPXES



In [ ]:
def meansquarerror(paramt,COPXtrain,masstrain,ASISXtrain,ASISYtrain,ASISZtrain):
    
    from sklearn.metrics import mean_squared_error
    
    A,alpha,beta, gama = paramt
        
    ytrue = COPXtrain
    ypredicted = centerofpressure(paramt,masstrain,ASISXtrain,ASISYtrain,ASISZtrain)
        
    
    
    return mean_squared_error(ytrue, ypredicted)
        


    

In [ ]:
from sklearn.model_selection import train_test_split
#from scipy import optmize
from scipy.optimize import minimize, rosen, rosen_der
first_guess = [1,1,1,1]
res = minimize(meansquarerror, first_guess,(COPXtrain,masstrain,ASISXtrain,ASISYtrain,ASISZtrain))
coeficientes = res.x
A=coeficientes[0]
alpha=coeficientes[1]
beta=coeficientes[2]
gama=coeficientes[3]

Test

In [ ]:

treinando=centerofpressure(coeficientes, masstrain,ASISXtrain,ASISYtrain,ASISZtrain) 

testando=centerofpressure(coeficientes, masstest,ASISXtest,ASISYtest,ASISZtest)


In [ ]:
time = np.arange(0,6000)/100
for i in range(0,len(ASISXtest)):
    plt.figure()
    plt.plot(time[:5970],detrend(testando[i,:]),color='purple',label='COP predicted')
    plt.plot(time[:5970],detrend(COPXreal[i]),color='blue', label='COP real')
    plt.title('COP real X COP calculated for test {:.1f}'.format(i))
    plt.xlabel('Time')
    plt.legend()
    plt.show()

In [ ]:
sennovo = sinx*A + ((1-A**2)**1/2)*cosx
print(np.max(sennovo))
print(np.min(sennovo))

In [ ]:
cosnovo = cosx*A - ((1-A**2)**1/2)*sinx
print(np.max(cosnovo))
print(np.min(cosnovo))